# Classification des données en utilisant le SVM

## Importation des données

In [ ]:
import pymongo
import sklearn as sk
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Tweet"]
user_collection = db["users_labeled"]

In [ ]:
users = list(user_collection.find({}))
users = pd.DataFrame(users)
users.index = users.user_id

#Normalement ce bloc va être supprimé une fois que le traitement aura été fait ailleurs
users = users.drop(columns=["_id","user_id","last_tweet_published_id","tweet_ids","friends_count","followers_count", "suspicious_fields", "suspicious_score"])
print(users.columns)

## Séparation des données labélisées en Apprentisage , Test et Validation

In [ ]:
Y=users.label
X=users.drop(columns=["label"])

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

## Classification avec SVM

### Déterminaison de la meilleure combinaison d'hyperparamètres

Dans un contexte de déséquilibre de classe, l'utilisation du rappel comme métrique de performance dans un modèle de classification binaire est justifiée. 

Le rappel mesure la capacité du modèle à trouver tous les échantillons positifs, ce qui est crucial lorsque les échantillons positifs sont rares ou représentent une classe minoritaire. 

En mettant l'accent sur la sensibilité à détecter les échantillons positifs, le rappel permet d'identifier les cas importants et de minimiser les faux négatifs, sans accorder une attention excessive aux faux positifs.


Selon la documentation de scikit learn:
> The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.


Cette métrique est alors la mieux placé pour évaluer les performances de notre modèle car nous cherchons à classer les utilisateurs en deux classes (atypique ou non) et que la majorité des utilisateurs ne l'est pas (présence d'une majorité négative).

Définition d'une fonction pour afficher une matrice de confusion

In [ ]:
def display_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    
    # Création de la figure
    fig, ax = plt.subplots()

    # Création de la heatmap
    heatmap = ax.imshow(cm, cmap='Blues')

    # Ajout des valeurs dans les cellules de la heatmap
    for i in range(len(cm)):
        for j in range(len(cm[i])):
            ax.text(j, i, cm[i][j], ha='center', va='center', color='black')

    # Définition des étiquettes des axes
    classes = ['Normal', 'Atypique']
    ax.set_xticks(range(len(classes)))
    ax.set_yticks(range(len(classes)))
    ax.set_xticklabels(classes)
    ax.set_yticklabels(classes)

    # Ajout d'une barre de couleur
    cbar = ax.figure.colorbar(heatmap, ax=ax)

    # Ajout des titres
    ax.set_xlabel('Prédictions')
    ax.set_ylabel('Vraies étiquettes')
    ax.set_title('Matrice de confusion')

    # Affichage de la figure
    plt.show()


Définition des hyperparamètres à essayer

In [ ]:
parameters = {
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'C': [1e-2, 1e-1, 1, 1e1],
    'gamma': ['scale', 'auto']
}

Instanciations

In [ ]:
svmc = SVC()
grille = GridSearchCV(estimator=svmc, param_grid=parameters, scoring='balanced_accuracy', cv=3, verbose=3)

Exécuter la recherche de grille pour trouver la meilleure configuration de modèle en ajustant les modèles sur les données d'apprentissage et en évaluant leur performance à l'aide de la validation croisée

In [ ]:
resultats = grille.fit(X_train, y_train)

Affichage du meilleur modèle

In [ ]:
print('Le meilleur modèle :', resultats.best_params_)

In [ ]:
resultats.cv_results_ 

In [ ]:
svm = resultats.best_estimator_
y_true = y_test
y_pred = svm.predict(X_test)

In [ ]:
display_confusion_matrix(y_true, y_pred)

In [ ]:
accuracy_score(y_true,y_pred)

In [ ]:
print(classification_report(y_true,y_pred))

On regarde si on aurait obtenu de meilleurs résultats/un autre meilleur kernel avec d'autres fonctions de scoring pour le GridSearch

In [ ]:
def test_scoring_value(scoring):
    print('>>> Scoring => ', scoring)
    # determination du meilleur kernel pour la fonction de scoring
    svmc = SVC()
    grille = GridSearchCV(estimator=svmc, param_grid=parameters, scoring=scoring, cv=2)
    resultats = grille.fit(X_train, y_train)
    print('Le meilleur modèle :', resultats.best_params_)
    
    # confusion matrix & accuracy
    svm = resultats.best_estimator_
    y_true = y_test
    y_pred = svm.predict(X_test)
    print(confusion_matrix(y_true, y_pred))
    print(accuracy_score(y_true,y_pred))

In [ ]:
scorings = ['accuracy', 'balanced_accuracy', 'top_k_accuracy', 'average_precision', 'neg_brier_score', 'f1', 'neg_log_loss', 'precision', 'recall', 'jaccard', 'roc_auc']
for s in scorings:
    test_scoring_value(s)

### Recherche des meilleurs attributs au vu de leur impact sur l'erreur

In [ ]:
attributs=users.columns
impact=[]
err_fix=1
verif=True
users_best=users

while (verif):
    for att in attributs :
        users_red=users_best.drop(columns=[att])
        Y=users_red.label
        X=users_red.drop(columns=["label"])

        X_tmp, X_test, Y_tmp, Y_test = train_test_split(X, Y, test_size=0.3)
        X_app, X_val, Y_app, Y_val = train_test_split(X_tmp, Y_tmp, test_size=0.5)
        svm=SVC(C=C_opti,kernel =kernel_opti)
        print(svm)
        svm.fit(X_app, Y_app)
        y_pred=svm.predict(X_test)
        #print(y_val)
        #print(y_pred)

        erreur=1-accuracy_score(Y_test,y_pred)

        print(erreur)
        impact.append({"attribut" :att,"erreur":erreur})

        mat=confusion_matrix(Y_test,y_pred)
        print(mat)
    #On retire l'attribut qui contribue le plus à  l'erreur 
    impact.sort(key=lambda x: x['erreur'])
    print(impact[0])
    attribut_neg=val[0]['attribut']
    err_cal=val[0]['erreur']
    users_best=users.drop(columns=[attribut_neg])

    attributs.remove(attribut_neg)
    if(err_cal>err_fix):
        verif=False
    else:
        err_fix=err_cal
    if(len(attributs)==2):
        verif=False


### ACP 

## Représentation graphique